In [15]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import requests
import time
import json
import keras
import warnings
import random
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from keras.optimizers import RMSprop, Adam, Adadelta, SGD
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras import backend as K

# Display preference
warnings.filterwarnings('ignore')

In [2]:
def scroll(driver, timeout):
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
    
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait
        time.sleep(timeout)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            
            break
        
        last_height = new_height   

In [3]:
# Set options
options = Options()
options.set_preference('permissions.default.image', 2)
options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', False)

def all_links(url, action, gender=0):
    
    # Setup driver
    driver = webdriver.Firefox(options=options, executable_path='D:/Users/bengriffith/Desktop/geckodriver.exe')

    # Wait before throwing an exception
    driver.implicitly_wait(20)

    # Open the page
    driver.get(url)
    
    # Start scrolling
    scroll(driver, 10)
    
    # Beautiful Soup parses page_source
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Close the driver
    driver.close()

    if action == 1:
    
        # Empty dictionary to store m from page source
        get_m = {}
    
        # Counter for get_m dictionary
        get_m_counter = 0

        # Looping through all the a elements in the page source
        for link in soup.find_all('a', {'class': 'iusc'}):
    
            # Add values to dictionary
            get_m[get_m_counter] = link.get('m')
        
            # Increment counter
            get_m_counter += 1
        
        # Convert from string to dictionary
        for key, value in get_m.items():
            get_m[key] = json.loads(value)
        
        # Empty list to store image links
        links = []
    
        # Loop through get_m dictionary and add image links to links list
        for value in get_m.values():
            links.append([value['murl'], gender])
        
        return links
    else:
        return soup

In [4]:
# List of names to search
names = []

In [5]:
# Popular names url
name_url = 'https://www.ssa.gov/oact/babynames/decades/century.html'

# Beautiful Soup object
soup_name = all_links(name_url, 0)

# Get td tag
td_tag = soup_name.select('.t-stripe > tbody > tr > td')

# Add name to search names
names += [name.get_text().lower() for name in td_tag if name.get_text().isalpha()]

In [6]:
# List for male names
male_names = []

# List for female names
female_names = []

for index, name in enumerate(names, 0):
    if index % 2 == 0:
        male_names.append([name, 0])
    else:
        female_names.append([name, 1])

In [13]:
# Get image links for each name
def get_links(names):
    
    image_links_name = []
    pre_image_links_count = 0
    post_image_links_count = 0
    
    for index in range(len(names)):
        
        # Update url
        url = 'https://www.bing.com/images/search?q=' + names[index][0] + '&qs=n&form=QBIR&qft=%20filterui%3Alicense-L2_L3_L4_L5_L6_L7%20filterui%3Aface-face&sp=-1&pq=' + names[index][0] + '&sc=8-9&sk=&cvid=8C2D8EC9A35A4948980A61669549C180'    
    
        # Number of image links before
        pre_image_links_count = len(image_links_name)
    
        # Add image links
        image_links_name += all_links(url, 1, names[index][1])
        
        # Number of image links after
        post_image_links_count = len(image_links_name)
        
        # Number of image links associated with name
        print('{} resulted in {} image links'.format(names[index][0], (post_image_links_count - pre_image_links_count)))
        
    return image_links_name

In [14]:
# List for image links
image_links = []

In [15]:
# Add male name image links
image_links += get_links(male_names)

james resulted in 175 image links
john resulted in 175 image links
robert resulted in 175 image links
michael resulted in 175 image links
william resulted in 175 image links
david resulted in 175 image links
richard resulted in 210 image links
joseph resulted in 175 image links
thomas resulted in 175 image links
charles resulted in 175 image links
christopher resulted in 175 image links
daniel resulted in 175 image links
matthew resulted in 175 image links
anthony resulted in 175 image links
donald resulted in 175 image links
mark resulted in 210 image links
paul resulted in 175 image links
steven resulted in 175 image links
andrew resulted in 175 image links
kenneth resulted in 175 image links
joshua resulted in 210 image links
george resulted in 175 image links
kevin resulted in 175 image links
brian resulted in 175 image links
edward resulted in 175 image links
ronald resulted in 175 image links
timothy resulted in 175 image links
jason resulted in 175 image links
jeffrey resulted i

In [16]:
# Add female name image links
image_links += get_links(female_names)

mary resulted in 175 image links
patricia resulted in 175 image links
jennifer resulted in 175 image links
linda resulted in 175 image links
elizabeth resulted in 175 image links
barbara resulted in 175 image links
susan resulted in 175 image links
jessica resulted in 175 image links
sarah resulted in 175 image links
karen resulted in 175 image links
nancy resulted in 175 image links
margaret resulted in 175 image links
lisa resulted in 175 image links
betty resulted in 175 image links
dorothy resulted in 175 image links
sandra resulted in 613 image links
ashley resulted in 175 image links
kimberly resulted in 175 image links
donna resulted in 175 image links
emily resulted in 175 image links
michelle resulted in 175 image links
carol resulted in 175 image links
amanda resulted in 175 image links
melissa resulted in 175 image links
deborah resulted in 175 image links
stephanie resulted in 175 image links
rebecca resulted in 175 image links
laura resulted in 175 image links
sharon resul

In [17]:
print('There are {} image links'.format(len(image_links)))

There are 36215 image links


In [18]:
# Shuffle image links list
random.shuffle(image_links)

In [19]:
# Header to accompany get request
headers = {'user-agent': 'scraper - school project (bengriffith@outlook.com)'}

# Counter to be used in image file name
image_counter = 0

# Counter for exceptions occurred
exception_counter = 0

# Write images to disk
for image_link in image_links:
    with open('images/face_' + str(image_counter) + '_' + str(image_link[1]) + '.png', 'wb') as f:
        try:
            response = requests.get(image_link[0], headers=headers)
            
            if response.ok:
                f.write(response.content)
        except:
            exception_counter += 1
            continue
    image_counter += 1
    
print('There were {} files removed due to a Request / Response Exception'.format(exception_counter))

There were 88 files removed due to a Request / Response Exception


In [3]:
images_directory = 'images/'

In [21]:
files_removed = 0

for filename in os.listdir(images_directory)[1:]:
    try:
        image = cv2.imread(images_directory + filename)
        (h, w) = image.shape[:2]
    except AttributeError:
        files_removed += 1
        os.remove(images_directory + filename)
        
print('There were {} files removed due to AttributeError'.format(files_removed))

There were 466 files removed due to AttributeError


In [5]:
prototxt = 'supporting/deploy.prototxt'
model = 'supporting/res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model)

In [8]:
# List to store detected face
data = []

# List to store labels
labels = []

In [9]:
len(os.listdir(images_directory))

35705

In [10]:
images_on_disk = os.listdir(images_directory)

In [ ]:
# Loop through five

# Add five to list

#

In [11]:
#files_skipped = 0

for i in range(1, len(os.listdir(images_directory))):
    
    # Image path
    image_path = images_directory + images_on_disk[i]
    
    # Load model
    #net = cv2.dnn.readNetFromCaffe(prototxt, model)

    # Load image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (96, 96))
    image = img_to_array(image)
    data.append(image)
    labels.append(images_on_disk[i][images_on_disk[i].find('.') - 1])
    #(h, w) = image.shape[:2]
    
    # Blob for the image
    #blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300))
    
    # Pass blob through network and retrieve detections / predictions
    #net.setInput(blob)
    #detections = net.forward()
    
    #for i in range(detections.shape[2]):
    
        # Get confidence associated with the prediction
        #confidence = detections[0, 0, i, 2]
    
        
        #if confidence > 0.3:
        
        # Coordinates for the bounding box
        #box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        #(startX, startY, endX, endY) = box.astype('int')
        
        # Coordinates for face
        #face = image[startY:endY, startX:endX]
        #try:
        #    face = cv2.resize(face, (96, 96))
        #except:
        #    files_skipped += 1
        #    continue
        
        #face = img_to_array(face)
        
        # Populate data and labels lists
        #data.append(face)
        #labels.append(images_on_disk[i][images_on_disk[i].find('.') - 1])
        #else:
        #    continue
        
#print('There were {} files skipped due to Resize Error'.format(files_skipped))

In [16]:
# Numpy arrays
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

# Binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=465)

In [22]:
def model(width, height, depth, classes):
    model = Sequential()
    input_shape = (height, width, depth)
    channel_dimension = -1
    
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dimension))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dimension))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dimension))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dimension))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dimension))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.45))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    
    return model

In [36]:
epochs = 20
batch_size = 32
image_dimensions = (96, 96, 3)

In [24]:
augment = ImageDataGenerator(rotation_range=25, vertical_flip=True, horizontal_flip=True, fill_mode='nearest')

In [32]:
gender_model = model(image_dimensions[1], image_dimensions[0], image_dimensions[2], 1)

In [33]:
gender_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
H = gender_model.fit(X_train, y_train, batch_size=batch_size, 
                     validation_data=(X_test, y_test), 
                     epochs=epochs, 
                     verbose=1)
score = gender_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 26778 samples, validate on 8926 samples
Epoch 1/20
26778/26778 [==============================] - 1755s 66ms/step - loss: 8.0855 - accuracy: 0.4928 - val_loss: 8.0444 - val_accuracy: 0.4954
Epoch 2/20
 7008/26778 [======>.......................] - ETA: 20:18 - loss: 8.1577 - accuracy: 0.4883

In [ ]:
# Train neural network to recognize whether male or female

# If female then choose female GAN

# If male then choose male GAN

# Will need to create / train female and male GANs